In [1]:
#%pip install steamreviews

In [1]:
import steamreviews

In [2]:
import json
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from textblob import TextBlob
from textstat.textstat import textstat
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

def load_data(app_id):
    # Data folder
    data_path = "data/"
    json_filename = "review_" + app_id + ".json"
    data_filename = data_path + json_filename
    with open(data_filename, 'r', encoding="utf8") as in_json_file:
        review_data = json.load(in_json_file)
    return review_data


def describe_data(review_data):
    try:
        query_summary = review_data['query_summary']
        sentence = 'Number of reviews: {0} ({1} up ; {2} down)'
        sentence = sentence.format(query_summary["total_reviews"], query_summary["total_positive"],
                                   query_summary["total_negative"])
    except KeyError:
        query_summary = None
        sentence = 'Query summary cannot be found in the JSON file.'
    print(sentence)
    reviews = list(review_data['reviews'].values())
    sentence = 'Number of downloaded reviews: ' + str(len(reviews))
    print(sentence)
    return query_summary, reviews


def aggregate_reviews(app_id):
    review_data = load_data(app_id)
    (_, reviews) = describe_data(review_data)
    review_stats = dict()
    ##
    # Review ID
    review_stats['recommendationid'] = []
    # Meta-data regarding the reviewers
    review_stats['num_games_owned'] = []
    review_stats['num_reviews'] = []
    review_stats['playtime_forever'] = []
    # Meta-data regarding the reviews themselves
    review_stats['language'] = []
    review_stats['voted_up'] = []
    review_stats['votes_up'] = []
    review_stats['votes_funny'] = []
    review_stats['weighted_vote_score'] = []
    review_stats['comment_count'] = []
    review_stats['steam_purchase'] = []
    review_stats['received_for_free'] = []

    # Stats regarding the reviews themselves
    review_stats['character_count'] = []
    review_stats['syllable_count'] = []
    review_stats['lexicon_count'] = []
    review_stats['sentence_count'] = []
    review_stats['difficult_words_count'] = []
    review_stats['flesch_reading_ease'] = []
    review_stats['dale_chall_readability_score'] = []

    # Sentiment analysis
    review_stats['polarity'] = []
    review_stats['subjectivity'] = []

    ##

    for review in reviews:
        review_content = review['review']

        # Review ID
        review_stats['recommendationid'].append(review["recommendationid"])

        # Meta-data regarding the reviewers
        review_stats['num_games_owned'].append(review['author']['num_games_owned'])
        review_stats['num_reviews'].append(review['author']['num_reviews'])
        review_stats['playtime_forever'].append(review['author']['playtime_forever'])

        # Meta-data regarding the reviews themselves
        review_stats['language'].append(review['language'])
        review_stats['voted_up'].append(review['voted_up'])
        review_stats['votes_up'].append(review['votes_up'])
        review_stats['votes_funny'].append(review['votes_funny'])
        review_stats['weighted_vote_score'].append(review['weighted_vote_score'])
        review_stats['comment_count'].append(review['comment_count'])
        review_stats['steam_purchase'].append(review['steam_purchase'])
        review_stats['received_for_free'].append(review['received_for_free'])

        # Stats regarding the reviews themselves
        review_stats['character_count'].append(len(review_content))
        review_stats['syllable_count'].append(textstat.syllable_count(review_content))
        review_stats['lexicon_count'].append(textstat.lexicon_count(review_content))
        review_stats['sentence_count'].append(textstat.sentence_count(review_content))
        review_stats['difficult_words_count'].append(textstat.difficult_words(review_content))
        try:
            review_stats['flesch_reading_ease'].append(textstat.flesch_reading_ease(review_content))
        except TypeError:
            review_stats['flesch_reading_ease'].append(None)
        review_stats['dale_chall_readability_score'].append(textstat.dale_chall_readability_score(review_content))

        # Sentiment analysis
        blob = TextBlob(review_content)
        review_stats['polarity'].append(blob.sentiment.polarity)
        review_stats['subjectivity'].append(blob.sentiment.subjectivity)

    return review_stats


def aggregate_reviews_to_pandas(app_id):
    review_stats = aggregate_reviews(app_id)

    df = pd.DataFrame(data=review_stats)

    # Correction for an inconsistency which I discovered when running df.mean(). These 2 columns did not appear in the
    # output of mean(). I don't think it has any real impact for clustering and other purposes, but just to be sure...
    if "comment_count" in df.columns:
        df["comment_count"] = df["comment_count"].astype('int')
    if "weighted_vote_score" in df.columns:
        df["weighted_vote_score"] = df["weighted_vote_score"].astype('float')

    return df


def find_top_languages_by_review_number(df, num_top_languages=3, verbose=True):
    # Extract a dataframe for reviews written in top languages (by review numbers)

    sorted_languages = df["language"].value_counts().index.tolist()

    top_languages = sorted_languages[0:num_top_languages]

    if verbose:
        print(top_languages)
        print(df["language"].describe())

    return top_languages


def extract_reviews_for_top_languages_only(df, top_languages, verbose=True):
    # Extract a dataframe for reviews written in top languages (by review numbers)

    s = pd.Series([lang in top_languages for lang in df["language"]], name='language')
    df_extracted = df[s.values]

    if verbose:
        print(df_extracted.groupby("language").mean())

    return df_extracted


def plot_univariate_distribution(data_frame, str_x="votes_up"):
    # Reference: https://seaborn.pydata.org/tutorial/distributions.html

    sns.distplot(data_frame[str_x], kde=False, fit=stats.lognorm)
    plt.show()

    return


def plot_box_plot(data_frame, str_x="language", str_y="votes_up"):
    # Reference: https://seaborn.pydata.org/examples/grouped_boxplot.html

    sns.boxplot(x=str_x, y=str_y, data=data_frame, palette="PRGn")
    plt.show()

    # NB: Discriminating between positive and negative reviews (with "voted_up") is not super useful in our case,
    # since we will only consider games with mostly positive reviews.

    return


def analyze_app_id(app_id, languages_to_extract=None, create_separate_plots=True):
    df = aggregate_reviews_to_pandas(app_id)

    num_top_languages = 3
    if languages_to_extract is None:
        top_languages = find_top_languages_by_review_number(df, num_top_languages)
    else:
        top_languages = languages_to_extract

    df_extracted = extract_reviews_for_top_languages_only(df, top_languages)

    # All the possible variables are listed here:
    variables = df_extracted.keys()
    print(variables)

    if create_separate_plots:
        variable_to_plot = "lexicon_count"
        plot_univariate_distribution(df_extracted, variable_to_plot)

        if num_top_languages > 1:
            plot_box_plot(df_extracted, "language", variable_to_plot)

    return df_extracted


def analyze_app_id_in_english(app_id):
    df = analyze_app_id(app_id, ['english'], False)
    return df


def get_review_content(app_id, review_id):
    data = load_data(app_id)

    reviews = list(data['reviews'].values())

    review_content = "-1"

    for review in reviews:
        if review['recommendationid'] == review_id:
            review_content = review['review']
            break

    return review_content


def plot_overlays_of_univariate_distribution(app_id_list, variable_to_plot="lexicon_count", languages_to_extract=None):
    # By definition, we want to overlay plots with this function, hence the following variable is set to False:
    if languages_to_extract is None:
        languages_to_extract = ['english']
    create_separate_plots = False

    current_palette = sns.color_palette(n_colors=len(app_id_list))

    for (iter_count, appID) in enumerate(app_id_list):
        print(appID)

        df = analyze_app_id(appID, languages_to_extract, create_separate_plots)

        sns.distplot(df[variable_to_plot], kde=False, fit=stats.beta,
                     color=current_palette[iter_count],
                     label=appID,
                     fit_kws={"label": appID + " fit", "color": current_palette[iter_count], "alpha": 0.25})

    plt.legend()
    plt.show()

    return


def main(argv):
    app_id_list = ["723090", "639780", "573170"]

    if len(argv) == 0:
        app_id = app_id_list[-1]
        print("No input detected. AppID automatically set to " + app_id)
        compare_app_ids_in_default_list = True
    else:
        app_id = argv[0]
        print("Input appID detected as " + app_id)
        compare_app_ids_in_default_list = False

    # Analyze one appID

    analyze_app_id(app_id)

    # Compare different appIDs

    if compare_app_ids_in_default_list:
        plot_overlays_of_univariate_distribution(app_id_list)


In [32]:
def download_reviews_for_app_id(app_id,
                                query_count=0,
                                chosen_request_params=None,
                                start_cursor='*',  # this could be useful to resume a failed download of older reviews
                                verbose=False):
    rate_limits = get_steam_api_rate_limits()

    request = dict(get_default_request_parameters(chosen_request_params))
    check_review_timestamp = bool('day_range' in request.keys() and request['filter'] != 'all')
    if check_review_timestamp:
        current_date = datetime.datetime.now()
        num_days = int(request['day_range'])
        date_threshold = current_date - datetime.timedelta(days=num_days)
        timestamp_threshold = datetime.datetime.timestamp(date_threshold)
        if verbose:
            if request['filter'] == 'updated':
                collection_keyword = 'edited'
            else:
                collection_keyword = 'first posted'
            print('Collecting reviews {} after {}'.format(collection_keyword,
                                                          date_threshold))

    review_dict = load_review_dict(app_id)

    previous_review_ids = set(review_dict['reviews'])

    num_reviews = None

    offset = 0
    cursor = start_cursor
    new_reviews = []
    new_review_ids = set()

    while (num_reviews is None) or (offset < num_reviews):

        if verbose:
            print('Cursor: {}'.format(cursor))

        success_flag, downloaded_reviews, query_summary, query_count, cursor = download_reviews_for_app_id_with_offset(
            app_id,
            query_count,
            cursor,
            chosen_request_params)

        delta_reviews = len(downloaded_reviews)

        offset += delta_reviews

        if success_flag and delta_reviews > 0:

            if check_review_timestamp:

                if request['filter'] == 'updated':
                    timestamp_str_field = 'timestamp_updated'
                else:
                    timestamp_str_field = 'timestamp_created'

                checked_reviews = list(filter(lambda x: x[timestamp_str_field] > timestamp_threshold,
                                              downloaded_reviews))

                delta_checked_reviews = len(checked_reviews)

                if delta_checked_reviews == 0:
                    if verbose:
                        print('Exiting the loop to query Steam API, because the timestamp threshold was reached.')
                    break
                else:
                    downloaded_reviews = checked_reviews

            new_reviews.extend(downloaded_reviews)

            downloaded_review_ids = [review['recommendationid'] for review in downloaded_reviews]

            # Detect full redundancy in the latest downloaded reviews
            if new_review_ids.issuperset(downloaded_review_ids):
                if verbose:
                    print('Exiting the loop to query Steam API, because this request only returned redundant reviews.')
                break
            else:
                new_review_ids = new_review_ids.union(downloaded_review_ids)

        else:
            if verbose:
                print('Exiting the loop to query Steam API, because this request failed.')
            break

        if num_reviews is None:
            review_dict['query_summary'] = query_summary
            # Initialize num_reviews with the correct value (this is crucial for the loop, do not change variable name):
            num_reviews = count(query_summary)
            # Also rely on num_reviews for display:
            print('[appID = {}] expected #reviews = {}'.format(app_id, num_reviews))

        if query_count >= rate_limits['max_num_queries']:
            cooldown_duration = rate_limits['cooldown']
            print('Number of queries {} reached. Cooldown: {} seconds'.format(query_count, cooldown_duration))
            time.sleep(cooldown_duration)
            query_count = 0

        if not previous_review_ids.isdisjoint(downloaded_review_ids):
            if verbose:
                print('Exiting the loop to query Steam API, because this request partially returned redundant reviews.')
            break

    # Keep track of the date (in string format) associated with the cursor at save time.
    review_dict['cursors'][str(cursor)] = time.asctime()

    for review in new_reviews:
        review_id = review['recommendationid']
        if review_id not in previous_review_ids:
            review_dict['reviews'][review_id] = review

    with open(get_output_filename(app_id), 'w') as f:
        f.write(json.dumps(review_dict) + '\n')

    return review_dict, query_count

In [98]:
#!pip install textstat
#!pip install textblob

In [25]:
request_params = dict()
#request_params['language'] = 'english'
#request_params['purchase_type'] = 'steam'
request_params['review_type'] = 'negative'



In [108]:
#request_params

{'language': 'english', 'purchase_type': 'steam'}

In [8]:
Hades = {}
Hades = steamreviews.download_reviews_for_app_id(1145360)

[appID = 1145360] expected #reviews = 13428


In [12]:
Hades

({'reviews': {'63884716': {'recommendationid': '63884716',
    'author': {'steamid': '76561198134936947',
     'num_games_owned': 26,
     'num_reviews': 1,
     'playtime_forever': 183,
     'playtime_last_two_weeks': 183,
     'last_played': 1582302721},
    'language': 'tchinese',
    'review': '玩著玩著就忘了時間了\n',
    'timestamp_created': 1582302754,
    'timestamp_updated': 1582302754,
    'voted_up': True,
    'votes_up': 0,
    'votes_funny': 0,
    'weighted_vote_score': 0,
    'comment_count': 0,
    'steam_purchase': True,
    'received_for_free': False,
    'written_during_early_access': True},
   '63883563': {'recommendationid': '63883563',
    'author': {'steamid': '76561198101091611',
     'num_games_owned': 25,
     'num_reviews': 4,
     'playtime_forever': 11504,
     'playtime_last_two_weeks': 2597,
     'last_played': 1582280678},
    'language': 'schinese',
    'review': '无论是我个人认为比较重要的打击感、难度曲线、爽快程度，还是比较次要的美术、音乐部分，都是佼佼者。\n\n建议最多改一下骰子，不要改资源，不然无法充分的体会通关的成就感。\n\n最后是一个吐槽：我是一个

In [37]:
Hades = {}
Hades, query_count = steamreviews.download_reviews_for_app_id(1145360,chosen_request_params=request_params)

NameError: name 'get_steam_api_rate_limits' is not defined

In [31]:
Hades

{'reviews': {'63884716': {'recommendationid': '63884716',
   'author': {'steamid': '76561198134936947',
    'num_games_owned': 26,
    'num_reviews': 1,
    'playtime_forever': 183,
    'playtime_last_two_weeks': 183,
    'last_played': 1582302721},
   'language': 'tchinese',
   'review': '玩著玩著就忘了時間了\n',
   'timestamp_created': 1582302754,
   'timestamp_updated': 1582302754,
   'voted_up': True,
   'votes_up': 0,
   'votes_funny': 0,
   'weighted_vote_score': 0,
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': True},
  '63883563': {'recommendationid': '63883563',
   'author': {'steamid': '76561198101091611',
    'num_games_owned': 25,
    'num_reviews': 4,
    'playtime_forever': 11504,
    'playtime_last_two_weeks': 2597,
    'last_played': 1582280678},
   'language': 'schinese',
   'review': '无论是我个人认为比较重要的打击感、难度曲线、爽快程度，还是比较次要的美术、音乐部分，都是佼佼者。\n\n建议最多改一下骰子，不要改资源，不然无法充分的体会通关的成就感。\n\n最后是一个吐槽：我是一个用手柄但是不喜欢开震动的玩家，请问什么时候能把放弃后再进

In [112]:
test = analyze_app_id_in_english('1145360')

Number of reviews: 13307 (13088 up ; 219 down)
Number of downloaded reviews: 13304
          num_games_owned  num_reviews  playtime_forever  voted_up  votes_up  \
language                                                                       
english        227.055744      9.78591       2329.079819   0.98869  1.025368   

          votes_funny  weighted_vote_score  comment_count  steam_purchase  \
language                                                                    
english      0.304573              0.09438       0.039263        0.950234   

          received_for_free  character_count  syllable_count  lexicon_count  \
language                                                                      
english            0.005978       254.602682       62.131039      45.005655   

          sentence_count  difficult_words_count  flesch_reading_ease  \
language                                                               
english         2.452416               7.328486            74.

In [113]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6189 entries, 1 to 13301
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   recommendationid              6189 non-null   object 
 1   num_games_owned               6189 non-null   int64  
 2   num_reviews                   6189 non-null   int64  
 3   playtime_forever              6189 non-null   int64  
 4   language                      6189 non-null   object 
 5   voted_up                      6189 non-null   bool   
 6   votes_up                      6189 non-null   int64  
 7   votes_funny                   6189 non-null   int64  
 8   weighted_vote_score           6189 non-null   float64
 9   comment_count                 6189 non-null   int32  
 10  steam_purchase                6189 non-null   bool   
 11  received_for_free             6189 non-null   bool   
 12  character_count               6189 non-null   int64  
 13  sy

In [33]:
#steamreviews.download_reviews_for_app_id(1145360,chosen_request_params=request_params)

In [34]:
#steamreviews.download_reviews_for_app_id(1145360)

In [19]:
#import json
#with open('test.json', 'w', encoding='utf-8') as f:
#    json.dump(Hades, f, ensure_ascii=False, indent=4)

In [35]:
#Hades[0]['reviews']

In [38]:
from pymongo import MongoClient
import requests
client = MongoClient()

In [89]:
#client.database_names()

<ipython-input-89-896185ff0974>:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  client.database_names()


['admin', 'books', 'config', 'local', 'outings', 'steamreviews']

In [33]:
#steam = client['Steam_Reviews']

In [91]:
db = client.steamreviews

In [93]:
#db.create_collection('steamreviews')

In [52]:
list(Hades[0]['reviews'].values())

[{'recommendationid': '63734469',
  'author': {'steamid': '76561198043117634',
   'num_games_owned': 100,
   'num_reviews': 1,
   'playtime_forever': 1489,
   'playtime_last_two_weeks': 911,
   'last_played': 1582047623},
  'language': 'schinese',
  'review': '非常好玩非常爽',
  'timestamp_created': 1582047706,
  'timestamp_updated': 1582047706,
  'voted_up': True,
  'votes_up': 0,
  'votes_funny': 0,
  'weighted_vote_score': 0,
  'comment_count': 0,
  'steam_purchase': True,
  'received_for_free': False,
  'written_during_early_access': True,
  '_id': ObjectId('5e4c7725af23768b990a3fbd')},
 {'recommendationid': '63731118',
  'author': {'steamid': '76561198352470998',
   'num_games_owned': 17,
   'num_reviews': 3,
   'playtime_forever': 970,
   'playtime_last_two_weeks': 849,
   'last_played': 1582048835},
  'language': 'english',
  'review': "There are so many things to like about Hades: the beautiful environments; the silky smooth animation of the characters; the 'just one more go' roguelit

In [71]:
#db.drop_collection('steamreviews')

{'nIndexesWas': 1, 'ns': 'steamreviews.steamreviews', 'ok': 1.0}

In [124]:
db.list_collection_names()

[]

In [74]:
dict_values = Hades[0]['reviews'].values()

In [85]:
len(dict_values)

13258

In [87]:
len(dict_values[1]a

15

In [105]:
dict_values[-1]

ObjectId('5e4c76b1af23768b990a3fbc')

In [103]:
#for item in dict_values:
#    print(type(item))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

In [106]:
db.steamreviews.insert_many(dict_values[:-1])

In [123]:
db.drop_collection('steamreviews')

{'nIndexesWas': 1, 'ns': 'steamreviews.steamreviews', 'ok': 1.0}